In [76]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import time

In [77]:
def work(browser, url):
    browser.get(url)
    ele_nums=[]
    time.sleep(10)
    js='window.scrollTo(0,document.body.scrollHeight);'
    browser.execute_script(js)
    time.sleep(5)
    browser.execute_script(js)
    try:
        for link in browser.find_elements_by_xpath("//*[@data-artnuml]"):
            if isContainClass(link.get_attribute('className'),'icon-pdf'):
                ele_num=link.get_attribute('data-artnum')
                ele_nums.append(ele_num)
        return ele_nums
    except:
        print("failure")

In [78]:
def isContainClass(allClass,targetClass):
    classArr=allClass.split(' ')
    result=False
    for str in classArr:
        if str==targetClass:
            result=Ture
            break
    return result

In [79]:
def getHtml(url):
    headers={"user-agent":"Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.111 Mobile Safari/537.36"}
    try:
        response=requests.get(url,timeout=40,headers=headers)
        response.raise_for_status()
        
        response.encoding=response.apparent_encoding
        
        return response.txt
    except:
        import traceback
        traceback.print_exc()

In [80]:
def getSoup(html):
    soup=BeautifulSoup(html,'html.parser')
    print(soup.body.find_all('a',attrs={'class':r'icon-pdf'}))

In [81]:
def downloadPaper(url):
    try:
        soup=BeautifulSoup(getHtml(url),'html.parser')
        result=soup.body.find_all('iframe')
        downloadUrl=result[-1].attrs['src'].split('?')[0]
        headers={'user-agent':'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.111 Mobile Safari/537.36'}
        response=requests.get(downliadUrl,timeout=80,headers=headers)
        fname=downloadUrl[-12:]
        print(fname)
        
        with open(fname,'ab+') as f:
            print('start download file',fname)
            f.write(response.content)
            
    except:
        import traceback
        with open('errorLog','ab+') as f:
            traceback.print_exc(file=f)

In [83]:
if __name__=='__main__':
    url='https://ieeexplore.ieee.org/search/searchresult.jsp?action=search&matchBoolean=true&queryText=(%22Document%20Title%22:computer%20science)&ranges=1980_2021_Year&highlight=true&returnFacets=ALL&returnType=SEARCH&matchPubs=true&refinements=ContentType:Journals'
    baseUrl='http://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber='
    maxPageNumber=3
    browser=webdriver.Chrome()
    if maxPageNumber>=1:
        eleNums=work(browser,url)
        eleNums=list(set(eleNums))
        for eleNum in eleNums:
            newUrl=baseUrl_str(eleNum)
            downloadPaper(newUrl)
    else:
        for i in range(2,maxPageNumber+1):
            url10=url+'&pageNumber='+str(i)
            eleNums=work(browser,url10)
            eleNums=list(set(eleNums))
            for eleNums in eleNums:
                newUrl=baseUrl+str(eleNum)
                downloadPaper(newUrl)